# 第 7 章　统计学与机器学习｜用 Python 动手学统计学

## 第 4 节　线性模型与神经网络

### 环境准备

In [2]:
# 用于数值计算的库
import numpy as np
import pandas as pd
import scipy as sp

# 用于估计统计模型的库 (部分版本会有警告信息)
import statsmodels.formula.api as smf
import statsmodels.api as sm

# 用于多层感知器的库
from sklearn.neural_network import MLPClassifier

# 导入示例数据
from sklearn.datasets import load_iris

# 区分训练集与测试集
from sklearn.model_selection import train_test_split

# 标准化数据
from sklearn.preprocessing import StandardScaler

# 设置浮点数打印精度
%precision 3

'%.3f'

### 读入数据并整形

In [3]:
# 导入示例数据
iris = load_iris()

In [4]:
# 解释变量的名称
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
# 响应变量的名称
iris.target_names

array(['setosa', 'versicolor', 'virginica'],
      dtype='<U10')

In [6]:
# 解释变量仅为萼片 (sepal)
X = iris.data[50:150, 0:2]
# 只取2种鸢尾花
y = iris.target[50:150]

print("解释变量行数与列数：", X.shape)
print("响应变量行数与列数：", y.shape)

解释变量行数与列数： (100, 2)
响应变量行数与列数： (100,)


In [7]:
# 把数据分为训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 2)

print("解释变量行数与列数：", X_train.shape)
print("响应变量行数与列数：", y_train.shape)

解释变量行数与列数： (75, 2)
响应变量行数与列数： (75,)


### 实现：logistic 回归

In [12]:
# 打印响应变量
y_train[0:10]

array([1, 1, 2, 2, 2, 2, 1, 1, 1, 1])

In [8]:
# 数据整形
# 解释变量的数据帧
X_train_df = pd.DataFrame(
    X_train, columns = ["sepal_len", "sepal_wid"])
# 响应变量的数据帧
y_train_df = pd.DataFrame({"species": y_train - 1})
# 连接数据帧
iris_train_df = pd.concat(
    [y_train_df, X_train_df], axis=1)
# 打印结果
print(iris_train_df.head(3))

   species  sepal_len  sepal_wid
0        0        5.7        2.8
1        0        6.6        3.0
2        1        6.1        3.0


In [17]:
# 模型化
# 长度与宽度模型
logi_mod_full = smf.glm(
    "species ~ sepal_len + sepal_wid", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# 长度模型
logi_mod_len = smf.glm(
    "species ~ sepal_len", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# 宽度模型
logi_mod_wid = smf.glm(
    "species ~ sepal_wid", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# 空模型
logi_mod_null = smf.glm(
    "species ~ 1", data = iris_train_df,
    family=sm.families.Binomial()).fit()

# 对比 AIC
print("full", logi_mod_full.aic.round(3))
print("len ", logi_mod_len.aic.round(3))
print("wid ", logi_mod_wid.aic.round(3))
print("null", logi_mod_null.aic.round(3))

full 76.813
len  76.234
wid  92.768
null 105.318


In [18]:
# 查看估计的系数等指标
logi_mod_len.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-16.4152,4.000,-4.104,0.000,-24.256,-8.575
sepal_len,2.6478,0.639,4.142,0.000,1.395,3.901


In [24]:
# 预测精度
# 数据整形
X_test_df = pd.DataFrame(
    X_test, columns = ["sepal_len", "sepal_wid"])

# 拟合与预测
logi_fit = logi_mod_len.fittedvalues.round(0)
logi_pred = logi_mod_len.predict(X_test_df).round(0)

# 正确数
true_train = sp.sum(logi_fit == (y_train - 1))
true_test = sp.sum(logi_pred == (y_test - 1))

# 命中率
result_train = true_train / len(y_train)
result_test = true_test / len(y_test)

# 打印结果
print("训练集的命中率：", result_train)
print("测试集的命中率：", result_test)

训练集的命中率： 0.746666666667
测试集的命中率： 0.68


### 实现：标准化

In [25]:
# 准备标准化
scaler = StandardScaler()
scaler.fit(X_train)
# 标准化
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
sp.std(X_train_scaled, axis=0)

array([ 1.,  1.])

In [27]:
sp.std(X_test_scaled, axis=0)

array([ 0.74 ,  0.679])

### 实现：神经网络

In [28]:
nnet = MLPClassifier(
    hidden_layer_sizes = (100,100),
    alpha = 0.07,
    max_iter = 10000,
    random_state = 0)
nnet.fit(X_train_scaled, y_train)

# 正确数
print("训练集的命中率：", nnet.score(X_train_scaled, y_train))
print("测试集的命中率：", nnet.score(X_test_scaled, y_test))

训练集的命中率： 0.893333333333
测试集的命中率： 0.68
